In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D
from keras.models import Sequential,load_model
import h5py

# Using ImageDataGenerator instead of Open-Cv as it helps OOM error 

In [3]:
train=ImageDataGenerator(rescale=1/255.0)
test=ImageDataGenerator(rescale=1/255.0)

In [4]:
gen_train=train.flow_from_directory("malaria\\",target_size=(100,100),class_mode="binary")
gen_test=test.flow_from_directory("test-malaria\\",target_size=(100,100),class_mode="binary")

Found 24546 images belonging to 2 classes.
Found 3012 images belonging to 2 classes.


In [8]:
gen_train.class_indices,gen_test.class_indices

({'Parasitized': 0, 'Uninfected': 1}, {'Parasitized': 0, 'Uninfected': 1})

# Model with 2 Conv Layer and 1 Dense Layer

In [10]:
model=Sequential()
model.add(Conv2D(64,(4,4),input_shape=(100,100,3),activation="relu",padding="SAME"))
model.add(MaxPooling2D((3,3)))
model.add(Conv2D(64,(4,4),activation="relu",padding="SAME"))
model.add(MaxPooling2D((3,3)))
model.add(Flatten())
model.add(Dense(128,activation="relu"))
model.add(Dropout(0.4))
model.add(Dense(1,activation="sigmoid"))

In [11]:
model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

In [12]:
model.fit_generator(gen_train,validation_data=gen_test,epochs=30)

Epoch 1/30
768/768 [==============================] - 482s 627ms/step - loss: 0.4266 - accuracy: 0.7889 - val_loss: 0.2016 - val_accuracy: 0.9436
Epoch 2/30
768/768 [==============================] - 28s 37ms/step - loss: 0.1601 - accuracy: 0.9492 - val_loss: 0.1623 - val_accuracy: 0.9472
Epoch 3/30
768/768 [==============================] - 29s 38ms/step - loss: 0.1360 - accuracy: 0.9552 - val_loss: 0.1543 - val_accuracy: 0.9522
Epoch 4/30
768/768 [==============================] - 51s 66ms/step - loss: 0.1224 - accuracy: 0.9587 - val_loss: 0.1652 - val_accuracy: 0.9489
Epoch 5/30
768/768 [==============================] - 94s 123ms/step - loss: 0.1105 - accuracy: 0.9639 - val_loss: 0.1504 - val_accuracy: 0.9545
Epoch 6/30
768/768 [==============================] - 94s 122ms/step - loss: 0.0996 - accuracy: 0.9658 - val_loss: 0.1674 - val_accuracy: 0.9475
Epoch 7/30
768/768 [==============================] - 94s 122ms/step - loss: 0.0856 - accuracy: 0.9706 - val_loss: 0.1746 - val_accu

KeyboardInterrupt: 

In [13]:
model.save("malaria_model.h5")

In [18]:
a=model.predict_classes(gen_test)

In [24]:
model.predict_classes(gen_test)

array([[1],
       [0],
       [0],
       ...,
       [0],
       [1],
       [1]])

In [15]:
model=load_model("malaria_model.h5")

In [17]:
gen_test.class_indices,gen_train.class_indices

({'Parasitized': 0, 'Uninfected': 1}, {'Parasitized': 0, 'Uninfected': 1})

In [86]:
import glob
filepath="test-malaria\\Uninfected\\C39P4thinF_original_IMG_20150622_111206_cell_75.png"

In [87]:
from keras.models import load_model
import cv2
import numpy as np

def preprocessing(i):
    data=cv2.imread(i)
    data=cv2.resize(data,(100,100))
    data=np.array(data)
    data=data.reshape(-1,100,100,3)
    return data/255.0

model=load_model("malaria_model.h5")
data=preprocessing(filepath)
s=model.predict_proba(data)
p=model.predict_classes(data)

if p==0:
    print("The cell is parasitized and the probability is :",1-s[0][0])
else :
    print("The cell is uninfected and probability is :",s[0][0])

The cell is uninfected and probability is : 0.5706284
